<a href="https://colab.research.google.com/github/stfc/janus-tutorials/blob/main/md.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Molecular Dynamics with MLIPs

## Setup environment (Optional)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!python3 -m pip install -U git+https://github.com/stfc/janus-core.git
!python3 -m pip install -U git+https://gitlab.com/ase/ase.git
!python3 -m pip install matgl
!python3 -m pip install chgnet

In [ ]:
from ase.visualize import view
from ase.build import bulk
import matplotlib.pyplot as plt
import numpy as np

from janus_core.calculations.single_point import SinglePoint
from janus_core.calculations.md import NPH, NPT, NVE, NVT, NVT_NH
from janus_core.helpers.stats import Stats

### Cooling

Build NaCl structure and attach the MACE calculator:

In [ ]:
NaCl = bulk('NaCl', 'rocksalt', a=5.63, cubic=True)
NaCl = NaCl * (2,2,2)

In [ ]:
view(NaCl)

Prepare a simulation, cooling the structure from 300K to 200K in stepx of 20K, with 50fs at each temperature:

In [ ]:
sp_NaCl_cooling = SinglePoint(struct=NaCl.copy(),
                 architecture="mace_mp",
                 device='cpu',
                 calc_kwargs={'model_paths':'small','default_dtype':'float64'})

cooling = NVT(
        struct=sp_NaCl_cooling.struct,
        temp_start=300.0,
        temp_end=200.0,
        temp_step=20,
        temp_time=50,
        stats_every=20,
)

Run cooling:

In [ ]:
cooling.run()

The final structures at each temperature are saved in `Cl32Na32-nvt-T300.0-final.xyz`, `Cl32Na32-nvt-T280.0-final.xyz`, ..., `Cl32Na32-nvt-T200.0-final.xyz`.

The statiscs from the simulation are also saved every 20 steps in `Cl32Na32-nvt-T300.0-T200.0-stats.dat`. This can then be analysed using the `Stats` module:

In [ ]:
data = Stats("Cl32Na32-nvt-T300.0-T200.0-stats.dat")

In [ ]:
print(data)

In [ ]:
plt.plot(data[0], data[5], label="Actual")
plt.plot(data[0], data[16], label="Target")
plt.legend()
plt.xlabel("Steps")
plt.ylabel("Temperature / K")
plt.show()

### Heating, followed by MD

This will prepare an NVT MD simulation, initially increasing the temperature from 0K to 300K in 20K steps, with 10fs at each temperature, before a further 100 steps (100fs) at 300K.

The final structure at each temperature will be saved, e.g. `Cl32Na32-nvt-T0-final.xyz`, `Cl32Na32-nvt-T0-final.xyz`, ..., `Cl32Na32-nvt-T300-final.xyz.`

In [ ]:
sp_NaCl_md = SinglePoint(struct=NaCl.copy(),
                 architecture="mace_mp",
                 device='cpu',
                 calc_kwargs={'model_paths':'small','default_dtype':'float64'})

heating = NVT(
        struct=sp_NaCl_md.struct,
        temp_start=0, # Start of temperature ramp
        temp_end=300.0, # End of temperature ramp
        temp_step=20, # Temperature ramp increments
        temp_time=10, # Time at each temperature in ramp
        temp=300, # MD temperature
        steps=100, # MD steps at 300K
)

In [ ]:
heating.run()

The same structure can then be used to run an additional NVE MD similation for 500 steps (0.5ps), with post-processing performed to compute the RDF by setting `post_process_kwargs = {"rdf_compute": True}`, with the results saved to `Cl32Na32-nve-rdf.dat`:

In [ ]:
md = NVE(
        struct=sp_NaCl_md.struct,
        temp=300,
        stats_every=5,
        steps=500,
        post_process_kwargs={"rdf_compute": True, "rdf_rmax": 5, "rdf_bins": 200}
)

In [ ]:
md.run()

In [ ]:
rdf = np.loadtxt("Cl32Na32-nve-rdf.dat")
bins, counts = zip(*rdf)

In [ ]:
plt.plot(bins, counts)
plt.ylabel("RDF")
plt.xlabel("Distance / Å")
plt.show()